# Implementing the Fundamental Functions

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import random
from shapely.geometry import box
import supervision as sv

%matplotlib inline

## 1. IoU (Intersection over Union) between two axis-aligned bounding boxes specified in the Ultralytics YOLO format

### Implementing IoU with Shapely and Supervision

In [29]:
def yolo_to_xyxy(bbox):
    x, y, w, h = bbox
    return [x - w / 2, y - h / 2, x + w / 2, y + h / 2]

# calculating IoU with shapely
def iou_shapely(box1, box2):
    # box takes (minx, miny, maxx, maxy), so unpack the values from yolo format
    box1_xyxy = box(*yolo_to_xyxy(box1))
    box2_xyxy = box(*yolo_to_xyxy(box2))

    intersection = box1_xyxy.intersection(box2_xyxy).area
    union = box1_xyxy.union(box2_xyxy).area

    return intersection / union if union > 0 else 0

# calculating IoU with supervision
def iou_supervision(box1, box2):
    # box_iou_batch takes a batch of boxes as a numpy array
    box1_xyxy = np.array([yolo_to_xyxy(box1)])
    box2_xyxy = np.array([yolo_to_xyxy(box2)])

    return sv.box_iou_batch(box1_xyxy, box2_xyxy)[0][0]

### Showing that IoU with Shapely and Supervision are equivalent

In [65]:
def generate_random_yolo_bbox(img_size=412, min_size=150, max_size=200):
    w = random.randint(min_size, max_size)
    h = random.randint(min_size, max_size)
    x = random.randint(w // 2, img_size - w // 2)
    y = random.randint(h // 2, img_size - h // 2)
    return (x, y, w, h)

def compare_iou(n_samples=10):
    for _ in range(n_samples):
        box1 = generate_random_yolo_bbox()
        box2 = generate_random_yolo_bbox()

        iou1 = iou_shapely(box1, box2)
        iou2 = iou_supervision(box1, box2)

        print(f"YOLO Box 1: {box1}, YOLO Box 2: {box2}")
        print(f"IoU (Shapely): {iou1:.4f}, IoU (Supervision): {iou2:.4f}")
        print(f"Difference: {abs(iou1 - iou2):.6f}")
        print("-" * 50)

compare_iou()

YOLO Box 1: (254, 96, 179, 170), YOLO Box 2: (247, 189, 178, 166)
IoU (Shapely): 0.2730, IoU (Supervision): 0.2730
Difference: 0.000000
--------------------------------------------------
YOLO Box 1: (316, 266, 157, 151), YOLO Box 2: (303, 254, 160, 181)
IoU (Shapely): 0.7157, IoU (Supervision): 0.7157
Difference: 0.000000
--------------------------------------------------
YOLO Box 1: (145, 289, 156, 188), YOLO Box 2: (293, 199, 164, 173)
IoU (Shapely): 0.0192, IoU (Supervision): 0.0192
Difference: 0.000000
--------------------------------------------------
YOLO Box 1: (298, 116, 198, 187), YOLO Box 2: (286, 281, 172, 200)
IoU (Shapely): 0.0737, IoU (Supervision): 0.0737
Difference: 0.000000
--------------------------------------------------
YOLO Box 1: (310, 325, 177, 150), YOLO Box 2: (209, 262, 179, 199)
IoU (Shapely): 0.1602, IoU (Supervision): 0.1602
Difference: 0.000000
--------------------------------------------------
YOLO Box 1: (190, 91, 171, 165), YOLO Box 2: (112, 272, 184, 

As we can see above, all the differences are zero. Therefore, both Shapely and Supervision implementations for IoU are equivalent.

## 2. Write a function to compute Average Precision (AP) 

### a. Use Pascal VOC 11 point interpolation method to implement the function 

### b. Use COCO 101-point interpolation method to implement the function

### c. Use Area under Precision-Recall Curve (AP) method to implement the function 

### d. Randomly generate 10 images of size 100x100. Randomly generate 10 ground truth boxes of size 20x20 and 10 predicted boxes of size 20x20 in each image. Assume there is only one class of objects. Compare the AP50 (Average Precision at IoU 0.5) computed by 3 of your methods